# Clusterlogs Notebook

In [1]:
import pandas as pd
from clusterlogs import pipeline
import numpy as np

## 1. Download data from file and create pandas DataFrame with index 

In [ ]:
df = pd.read_csv('samples/harvester_errors24.csv', sep=';')

In [ ]:
df = pd.read_csv('/Users/maria/cernbox/LogsClusterization/Harvester/data_sample.csv', sep='\t')

In [10]:
df = pd.read_csv('/Users/maria/cernbox/LogsClusterization/Harvester/data_sample-2020-02-27 15_01_06.750914.csv', sep='\t')

In [2]:
df = pd.read_csv('/Users/maria/cernbox/LogsClusterization/Harvester/data_sample_superror-2020-03-03 09_35_47.158324.csv', sep='\t')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [3]:
target = 'message'

In [4]:
len(np.unique(df['message'].values))

161

## 2. Execute clusterization pipeline

In [ ]:
cluster = pipeline.Chain(df, target, mode='create', model_name='models/harvester_new.model', matching_accuracy=0.8)

In [13]:
cluster = pipeline.Chain(df, target, mode='process', model_name='models/harvester_30days.model', matching_accuracy=0.8, clustering_type='ML')

In [5]:
cluster = pipeline.Chain(df, target, mode='create', model_name='models/harvester_new.model', matching_accuracy=0.8, clustering_type='ML')

In [6]:
cluster.process()

Found 68 equal groups
Vectorization of tokens finished
Vectorization of sentences is finished
Vocabulary size = 190
Number of dimensions is 14


/Users/maria/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


DBSCAN finished with 25 clusters


In [ ]:
df['sequence'].values

In [ ]:
cluster.groups.shape

## 3. Result: all clusters (big clusters and outliers) - sorted by cluster size 

In [7]:
cluster.result

,pattern,sequence,tokenized_pattern,indices,cluster_size,common_phrases
14,Diag from worker : Condor HoldReason: ;...,"[Diag, from, worker, Condor, HoldReason, Worke...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Condor, ▁,...","[0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 1...",286,"[(authorization failure worker canceled, 14.66..."
0,Diag from worker : Failed in data staging: Fai...,"[Diag, from, worker, Failed, in, data, staging...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Failed, ▁,...","[158, 227, 245, 318, 332, 335, 148, 128, 130, ...",117,"[(data staging failed, 8.524590163934427), (wo..."
10,Diag from worker : LRMS error: (271) job killed:,"[Diag, from, worker, LRMS, error, job, killed]","[Diag, ▁, from, ▁, worker, ▁, :, ▁, LRMS, ▁, e...","[34, 348, 356, 357, 358, 359, 361, 362, 363, 3...",104,"[(diag, 1.0)]"
9,Diag from worker : Condor HoldReason: error ;...,"[Diag, from, worker, Condor, HoldReason, error...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Condor, ▁,...","[372, 704, 759, 760, 761, 762, 763, 764, 765, ...",76,"[(sigxcpu reason worker canceled, 14.5), (sigt..."
18,Diag from worker : LRMS error: (271) Job timeout,"[Diag, from, worker, LRMS, error, Job, timeout]","[Diag, ▁, from, ▁, worker, ▁, :, ▁, LRMS, ▁, e...","[1, 83, 87, 95, 144, 175, 214, 225, 226, 228, ...",70,"[(worker lrms error job timeout, 25.0), (diag,..."
8,Diag from worker : Condor HoldReason: None ; C...,"[Diag, from, worker, Condor, HoldReason, None,...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Condor, ▁,...","[8, 86, 111, 211, 464, 465, 466, 467, 468, 475...",58,"[(worker condor holdreason, 9.0), (condor remo..."
2,Diag from worker : LRMS error: (-1) Job missin...,"[Diag, from, worker, LRMS, error, Job, missing...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, LRMS, ▁, e...","[39, 40, 41, 43, 45, 46, 50, 52, 85, 114, 684,...",30,"[(worker lrms error job missing, 25.0), (diag,..."
20,Diag from worker : Condor HoldReason: ; Wor...,"[Diag, from, worker, Condor, HoldReason, Worke...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Condor, ▁,...","[13, 433, 435, 443, 738, 739, 740, 741, 781, 7...",14,"[(worker condor holdreason job, 15.5), (found ..."
11,Diag from worker : Condor HoldReason: Network ...,"[Diag, from, worker, Condor, HoldReason, Netwo...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Condor, ▁,...","[135, 163, 165, 174, 197, 200, 201, 206, 215, ...",12,"[(authorization failure, 4.0), (diag, 1.0), (s..."
22,Diag from worker : Job submission to LRMS failed,"[Diag, from, worker, Job, submission, to, LRMS...","[Diag, ▁, from, ▁, worker, ▁, :, ▁, Job, ▁, su...","[81, 131, 146, 374, 471, 613, 692, 757]",8,"[(worker job submission, 9.0), (lrms failed, 4..."


In [ ]:
cluster.in_cluster(cluster.result, 2)

### Print only patterns

In [16]:
cluster.result['pattern'].values

array(['  ', 'LRMS error: (271) job killed: ',
       'LRMS error: (-1) Job missing from SLURM',
       'LRMS error: (-1) Job missing from SLURM; Failed in data staging: Failed to prepare destination srm://srm.ndgf.org:8443/srm/managerv2?SFN=/atlas/disk/atlasdatadisk/rucio/mc16_13TeV//.:checksumtype=adler32:',
       'LRMS error: (271) Job timeout',
       "Failed to create a JOB; HTTPSConnectionPool(host='', port=6443): Max retries exceeded with url: /apis/batch/v1/namespaces//jobs (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at >: Failed to establish a new connection: [Errno ] Connection ',))",
       'Killed by Harvester due to worker queuing too long.',
       'LRMS error: (-1) RemoveReason: Job removed by SYSTEM_PERIODIC_REMOVE due to exceeding the requested memory.',
       'Error reading user generated output file list',
       'LRMS error: (271) Job was cancelled'], dtype=object)

In [21]:
cluster.result['common_phrases'].values

array([list([('payload execution error returned', 16.0)]),
       list([('job restarted undesirably', 9.0), ('condor removereason removed', 8.5), ('condor holdreason', 4.5), ('system_periodic_remove due', 4.0)]),
       list([('submitted due', 4.0), ('incomplete data', 4.0), ('worker', 1.0)]),
       list([('harvester due', 4.0), ('held', 1.0), ('long', 1.0), ('found', 1.0)]),
       list([('harvester due', 4.0), ('held', 1.0), ('long', 1.0), ('found', 1.0)]),
       list([('condor removereason python initiated action', 23.5), ('condor holdreason', 5.5), ('user atlpan', 4.0)]),
       list([('condor holdreason cream error job', 25.0), ('sigxcpu reason worker canceled', 14.5), ('sigterm reason worker canceled', 14.5), ('sigxcpu worker canceled', 10.5), ('sigterm worker canceled', 10.5), ('harvester due', 4.0), ('terminated', 1.0), ('held', 1.0), ('long', 1.0), ('found', 1.0)]),
       list([('moment faultcause submissions', 9.0), ('cream service', 4.0), ('accept jobs', 4.0)]),
       li

In [ ]:
cluster.in_cluster(cluster.result, 43)

### Split clusters to big (cluster_size >= 1000) and small (cluster_size < 1000)

In [ ]:
big, small = cluster.split_clusters(cluster.result, 'cluster_size', 1000)

In [ ]:
big['pattern'].values

In [ ]:
small

### Print all messages from cluster #40

In [ ]:
cluster.in_cluster(clusters, 40)

### Display the performance of all stages

In [ ]:
cluster.timings

In [ ]:
from gensim.summarization import keywords
text = '''Challenges in natural language processing frequently involve
speech recognition, natural language understanding, natural language
generation (frequently from formal, machine-readable logical forms),
connecting language and machine perception, dialog systems, or some
combination thereof.'''

In [ ]:
import spacy
import pytextrank

In [ ]:
nlp = spacy.load("en_core_web_sm")
tr = pytextrank.TextRank()

In [ ]:
phrases = Phrases(text, min_count=1, threshold=1, scoring='npmi', delimiter=b' ')

In [ ]:
nlp = spacy.load("en_core_web_sm")
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
doc = nlp(text)
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)

In [ ]:
spacy.load('en_core_web_sm')

In [ ]:
text = """Automatic summarization is the process of reducing a text document with a \
computer program in order to create a summary that retains the most important points \
of the original document. As the problem of information overload has grown, and as \
the quantity of data has increased, so has interest in automatic summarization. \
Technologies that can make a coherent summary take into account variables such as \
length, writing style and syntax. An example of the use of summarization technology \
is search engines such as Google. Document summarization is another."""
text1 = """
Error on the surl while putdone. This SURL does not exist in the original request. 
"""

from summa import summarizer
print(summarizer.summarize(text1))

In [ ]:
from summa import keywords

In [ ]:
print(keywords.keywords(text1))

In [ ]:
import regroup

In [ ]:
def calc_cache_pos(strings, indexes):
    factor = 1
    pos = 0
    for s, i in zip(strings, indexes):
        pos += i * factor
        factor *= len(s)
    return pos

def lcs_back(strings, indexes, cache):
    if -1 in indexes:
        return ""
    match = all(strings[0][indexes[0]] == s[i]
                for s, i in zip(strings, indexes))
    if match:
        new_indexes = [i - 1 for i in indexes]
        result = lcs_back(strings, new_indexes, cache) + strings[0][indexes[0]]
    else:
        substrings = [""] * len(strings)
        for n in range(len(strings)):
            if indexes[n] > 0:
                new_indexes = indexes[:]
                new_indexes[n] -= 1
                cache_pos = calc_cache_pos(strings, new_indexes)
                if cache[cache_pos] is None:
                    substrings[n] = lcs_back(strings, new_indexes, cache)
                else:
                    substrings[n] = cache[cache_pos]
        result = max(substrings, key=len)
    cache[calc_cache_pos(strings, indexes)] = result
    return result

def lcs(strings):
    """
    >>> lcs(['666222054263314443712', '5432127413542377777', '6664664565464057425'])
    '54442'
    >>> lcs(['abacbdab', 'bdcaba', 'cbacaa'])
    'baa'
    """
    if len(strings) == 0:
        return ""
    elif len(strings) == 1:
        return strings[0]
    else:
        cache_size = 1
        for s in strings:
            cache_size *= len(s)
        cache = [None] * cache_size
        indexes = [len(s) - 1 for s in strings]
        return lcs_back(strings, indexes, cache)

In [ ]:
lcs(['abacbdab', 'bdcaba', 'cbacaa', 'abacbdabaa'])

In [ ]:
import nltk 
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)